# Imports and Commons

In [4]:
%load_ext autoreload
%autoreload 2
import os
import re
import json
import copy
import sys
from tqdm import tqdm
from pprint import pprint

import matplotlib.pyplot as plt
import seaborn as sns

import pytrec_eval
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
from haystack.pipelines import Pipeline
from haystack.nodes import BM25Retriever, ElasticsearchRetriever
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import BM25Retriever, SentenceTransformersRanker
from haystack.nodes.reader import FARMReader
from haystack.utils import print_answers
import warnings
warnings.filterwarnings('ignore')

sys.path.append('../../')
import globals
from elastic_search_utils import elastic_utils
from haystack_utils.retrievers import BioASQ_Retriever
import bioasq_eval
import haystack_util

working_folder = globals.PATH.home + '/data/working_folder'
eval_home = globals.PATH.eval_home + '/'
gs_google_docs = eval_home + '/examples/aueb_google_docs/aueb_nlp-bioasq6b-submissions/'
index_name = globals.BIOASQ.index + 'working_folder'
model_id = 'doc_retrieval_test'

es = Elasticsearch(globals.ES.server)

# set document store
document_store = ElasticsearchDocumentStore()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Set Eval or Prediction

In [5]:
#evaluate over aueb documents
test_batch_docs = [ #('','8b5_ES_30_full.json')
                ('6B1_golden.json', gs_google_docs+'1-aueb-nlp-4.json'),
                ('6B2_golden.json', gs_google_docs+'2-aueb-nlp-4.json'),
                ('6B3_golden.json', gs_google_docs+'3-aueb-nlp-4.json'),
                ('6B4_golden.json', gs_google_docs+'4-aueb-nlp-4.json'),
                ('6B5_golden.json', gs_google_docs+'5-aueb-nlp-4.json')
               ]

In [6]:
test_batch_doc = f'{working_folder}/test11b/BioASQ-task11bPhaseA-testset1.json'
test_batch_json = json.load(open(test_batch_doc))

In [7]:
is_eval = True

# Document Retrieval

## BM25

In [5]:
# create the retriever
retriever = BioASQ_Retriever(document_store = document_store)
# create the Query Pipeline
bm25_pipeline = Pipeline()
bm25_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])

### Evaluate Results

In [6]:
pipeline_params = {"Retriever": {"top_k": 10}}
docs_scores, pass_scores = haystack_util.evaluate_batch_list(test_batch_docs, bm25_pipeline, pipeline_params, 'bm25', max_num_docs = 10, max_num_passages = 10)
#df.to_csv(working_folder + "/" + model_id+'.csv', index=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:23<00:00,  4.34it/s]


In [7]:
display(docs_scores)

,Batch,Method,Mean precision,Recall,F-Measure,MAP,GMAP
0,1-aueb-nlp-4.json,bm25,0.186,0.4315,0.2129,0.1143,0.0118
1,2-aueb-nlp-4.json,bm25,0.175,0.4186,0.1948,0.1105,0.0098
2,3-aueb-nlp-4.json,bm25,0.198,0.4364,0.2037,0.1209,0.0170
3,4-aueb-nlp-4.json,bm25,0.135,0.3880,0.1570,0.0846,0.0033
4,5-aueb-nlp-4.json,bm25,0.114,0.2074,0.1232,0.0606,0.0011


In [8]:
display(pass_scores)

,Batch,Method,Mean precision,Recall,F-Measure,MAP,GMAP
0,1-aueb-nlp-4.json,bm25,0.2127,0.2501,0.1924,0.1659,0.0111
1,2-aueb-nlp-4.json,bm25,0.2873,0.2146,0.1850,0.2337,0.0231
2,3-aueb-nlp-4.json,bm25,0.2600,0.2607,0.2298,0.2306,0.0359
3,4-aueb-nlp-4.json,bm25,0.1761,0.2367,0.1646,0.1479,0.0033
4,5-aueb-nlp-4.json,bm25,0.1409,0.1849,0.1285,0.1007,0.0024


## BM25 with 100 documents

In [9]:
# create the retriever
retriever = BioASQ_Retriever(document_store = document_store)
# create the Query Pipeline
bm25_pipeline = Pipeline()
bm25_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
# run the pipeline
pipeline_params = {"Retriever": {"top_k": 100}}
docs_scores, pass_scores = haystack_util.evaluate_batch_list(test_batch_docs, bm25_pipeline, pipeline_params, method_id = 'bm25', max_num_docs = 100, max_num_passages = 10)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:39<00:00,  2.54it/s]


In [10]:
display(docs_scores)

,Batch,Method,Mean precision,Recall,F-Measure,MAP,GMAP
0,1-aueb-nlp-4.json,bm25,0.0406,0.7396,0.0741,0.1657,0.0469
1,2-aueb-nlp-4.json,bm25,0.0432,0.7236,0.0772,0.1710,0.0381
2,3-aueb-nlp-4.json,bm25,0.0509,0.7248,0.0887,0.1877,0.0621
3,4-aueb-nlp-4.json,bm25,0.0333,0.6672,0.0601,0.1254,0.0119
4,5-aueb-nlp-4.json,bm25,0.0364,0.5822,0.0656,0.1038,0.0111


## BM25 + Ranker

In [11]:
retriever = BioASQ_Retriever(document_store = document_store)
# create the Sentence Transformer Ranker
#sentence-transformers/distilbert-base-nli-stsb-quora-ranking
#ranker = SentenceTransformersRanker(model_name_or_path="sentence-transformers/distilbert-base-nli-stsb-quora-ranking")
ranker = SentenceTransformersRanker(model_name_or_path="cross-encoder/ms-marco-MiniLM-L-12-v2")

# create the Query Pipeline
pipeline_bm25_ranker = Pipeline()

# add bm25 retriever
pipeline_bm25_ranker.add_node(component=retriever, name="BM25Retriever", inputs=["Query"])
pipeline_bm25_ranker.add_node(component=ranker, name="Ranker", inputs=["BM25Retriever"])

In [ ]:
pipeline_params = {"BM25Retriever": {"top_k": 100}}
docs_scores, pass_scores = haystack_util.evaluate_batch_list(test_batch_docs, pipeline_bm25_ranker, pipeline_params, method_id = 'bm25', max_num_docs = 100, max_num_passages = 10)

 79%|███████████████████████████████████████████████████████████████████████████████████▋                      | 79/100 [04:30<01:12,  3.44s/it]

In [ ]:
pipeline_params = {"Retriever": {"top_k": 10}}
docs_scores, pass_scores = haystack_util.evaluate_batch_list(test_batch_docs, bm25_pipeline, pipeline_params, 'bm25')

# run the pipeline
prediction = pipeline.run(query="covid", params=)

# predict
print([p.id for p in prediction['documents']])